In [1]:
# Train the Vit using Food101 dataset 

In [2]:
%%capture 
%run config.ipynb
%run data-install.ipynb
%run ViT.ipynb

In [3]:
from torch.optim import Adam 
from datetime import datetime 
import torch 
from tqdm import tqdm

In [4]:
model = ViT(
    image_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=IN_CHANNELS,
    n_head=N_HEAD, 
    d_model=D_MODEL, 
    ffn_hidden=FFN_HIDDEN, 
    mlp_hidden=MLP_HIDDEN, 
    n_layers=N_LAYERS, 
    class_num=CLASS_NUM, 
    device=device, 
    drop_prob=DROP_PROB,
)

def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad) 

logger.info(f'model parameter #: {count_parameters(model)}')

In [5]:
# Setup optimizer 
optimizer = Adam(params = model.parameters(), lr=INIT_LR)

# Setup loss function for training 
loss_func = nn.CrossEntropyLoss()

In [6]:
train_loss_per_epoch = []
test_loss_per_epoch = []

In [7]:
def train_epoch(epoch_num): 
    model.train()
    train_epoch_loss = 0 

    for step, (img, food) in tqdm(enumerate(train_dataloader)):
        img = img.to(device)
        food = food.to(device)
        out, _ = model(img)

        print(f'img shape: {img.shape}')
        print(f'out shape: {out.shape}')
        print(f'food shape: {food.shape}')


        loss = loss_func(out, food)
        print(f'loss: {loss}')

        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        train_epoch_loss += loss.item()

        if step % 10 == 0: 
            logger.info(f'EPOCH #{epoch_num} STEP #{step} | loss: {loss.item()}, avg_loss: {train_epoch_loss / (step + 1)}')

    train_step_loss = train_epoch_loss / (step + 1) 
    return train_epoch_loss 

In [8]:
def evaluate(): 
    model.eval()
    test_epoch_loss = 0 

    with torch.no_grad(): 
        for step, (img, food) in tqdm(enumerate(test_dataloader)):
            img = img.to(device)
            food = food.to(device)
            out, _ = model(img)

            loss = loss_func(out, food)

            test_epoch_loss += loss.item()

    test_step_loss = test_epoch_loss / (step + 1)

    return test_epoch_loss

In [9]:
# Actual training is done here 

for epoch in range(EPOCHS): 
    train_epoch_loss, train_step_loss = train_epoch(epoch)
    test_epoch_loss, test_step_loss = evaluate()

    train_loss_per_epoch.append(train_epoch_loss)
    test_loss_per_epoch.append(test_epoch_loss)

    logger.info(f'Epoch #{epoch} End | Train Loss: {train_epoch_loss} | Test Loss: {test_epoch_loss}')

    model_path = model_dir / f'model_{timestamp}_{epoch}'
    torch.save(model.state_dict(), model_path)

logger.info(f'Training Completely Ended!!')
    

0it [00:00, ?it/s]

img shape: torch.Size([16, 3, 256, 256])
out shape: torch.Size([16, 101])
food shape: torch.Size([16])
loss: 4.703150749206543


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.